In [14]:
!pip install scrapy

In [15]:
# import the libraries 
import pandas as pd 
import numpy as np
import requests
from scrapy import Selector

In [3]:
# URL of the first website
url = 'https://www.abs.gov.au/statistics/health/health-conditions-and-risks/smoking-and-vaping/latest-release'
response = requests.get(url)
response.raise_for_status()
html_content = response.content 

In [17]:
full_selector = Selector(text=html_content)
table = full_selector.xpath('//table[@class="responsive-enabled"]')
rows = table[1].xpath('.//thead//tr | .//tbody//tr')

/home/ubuntu/anaconda3/lib/python3.11/site-packages/scrapy/selector/unified.py:83: UserWarning: Selector got both text and root, root is being ignored.
  super().__init__(text=text, type=st, root=root, **kwargs)


In [18]:
from typing import List
import re

def parse_row(row: Selector) -> List[str]:
    cells = row.xpath('.//th | .//td')
    row_data = []

    for cell in cells:
        cell_text = cell.xpath('.//text()').getall()
        cell_text = ' '.join(cell_text)  # Join all text nodes within the cell
        cell_text = cell_text.strip()  # Strip whitespace from the beginning and end
        cell_text = re.sub(r'\s+', ' ', cell_text)  # Replace multiple spaces with a single space
        cell_text = re.sub(r'<.*?>', ' ', cell_text)  # Remove remaining HTML tags
        cell_text = cell_text.replace('\xa0', ' ')  # Replace non-breaking space with regular space
        row_data.append(cell_text)

    return row_data

In [20]:
header = parse_row(rows[0])
table_data = [parse_row(row) for row in rows[1:]]
smoking_df = pd.DataFrame(table_data, columns=header)
smoking_df.head(8)

,,2011–12 (%),95% confidence interval (%) (low),95% confidence interval (%) (high),2014–15 (%),95% confidence interval (%) (low),95% confidence interval (%) (high),2017–18 (%),95% confidence interval (%) (low),95% confidence interval (%) (high),2022 (%),95% confidence interval (%) (low),95% confidence interval (%) (high)
0,15–17,4.2,2.7,5.7,2.7,1.3,4.1,1.9,1.1,2.7,1.6,0.3,2.9
1,18–24,16.6,14.5,18.7,14.2,11.8,16.6,13.9,12.5,15.3,7.3,5.4,9.2
2,25–34,19.5,17.9,21.1,17.3,15.0,19.6,14.7,13.5,15.9,10.9,9.1,12.7
3,35–44,18.9,17.5,20.3,16.6,14.9,18.3,16.0,14.8,17.2,10.9,9.2,12.6
4,45–54,20.2,18.8,21.6,17.7,15.7,19.7,16.9,15.7,18.1,13.8,11.9,15.7
5,55–64,14.2,13.0,15.4,14.6,12.9,16.3,15.2,14.1,16.3,14.9,13.0,16.8
6,65–74,8.9,7.5,10.3,8.8,7.1,10.5,8.7,7.9,9.5,8.7,7.1,10.3
7,75 years and over,3.9,3.0,4.8,3.6,2.3,4.9,4.5,3.8,5.2,2.9,1.4,4.4


In [21]:
# URL of the second website
url2 = 'https://www.cdc.gov/tobacco/data_statistics/fact_sheets/adult_data/cig_smoking/index.htm'

response = requests.get(url2)
response.raise_for_status()
html_content = response.content


In [22]:
text = Selector(text=html_content)
div_sex = text.xpath('//div[contains(@class, "card-body") and .//p[contains(text(), "Current cigarette smoking was higher among men than women")]]/ul')
by_sex_list = div_sex.xpath('.//li/text()')
by_sex_list.getall()

/home/ubuntu/anaconda3/lib/python3.11/site-packages/scrapy/selector/unified.py:83: UserWarning: Selector got both text and root, root is being ignored.
  super().__init__(text=text, type=st, root=root, **kwargs)


['About 13 of every 100 adult men (13.1%)',
 'About 10 of every 100 adult women (10.1%)']

In [23]:
div_age = text.xpath('//div[contains(@class, "card-body") and .//p[contains(text(), "Current cigarette smoking was highest among people aged 25–44 years and 45–64 years. Current cigarette smoking was lowest among people aged 18-24 years")]]/ul')
by_age_list = div_age.xpath('.//li/text()')
by_age_list.getall()

['About 5 of every 100 adults aged 18–24 years (5.3%)',
 'Nearly 13 of every 100 adults aged 25–44 years (12.6%)',
 'Nearly 15 of every 100 adults aged 45–64 years (14.9%)',
 'About 8 of every 100 adults aged 65 years and older (8.3%)']